In [1]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
from matplotlib import pyplot as plt
import time

In [2]:
working_dir = "/home/fvalle/phd/datasets/breast"
os.chdir(working_dir)
dirs = os.listdir("data")

In [3]:
len(dirs)

1223

In [5]:
df_genes = pd.read_csv("genes.txt", index_col=[0])
protein_coding = df_genes[df_genes['type_of_gene']=='protein-coding']
#mirna = df_genes[df_genes['type_of_gene']=='microRNA']
select_genes = protein_coding.index.values
#select_genes = mirna.index.values

In [6]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=[0], index_col=[0])
#print(len(df['gene']))
df = df.loc[select_genes,:]
df.head()

,lenght,type_of_gene,exons
ENSG00000167578,18722.0,protein-coding,18696.0
ENSG00000078237,46830.0,protein-coding,38812.0
ENSG00000146083,11328.0,protein-coding,10691.0
ENSG00000158486,226330.0,protein-coding,217697.0
ENSG00000198242,4966.0,protein-coding,4375.0


In [ ]:
#fpkm
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM]{5}.txt[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [7]:
#counts
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.htseq]{6}[\.\_counts]{7}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile
    print("Nothing found")
    print(os.listdir(directory))

In [ ]:
#fpkm-uq
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.FPKM-UQ]{0,8}[\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [ ]:
##miRNA
def getFilenameFromDir(directory):
    for element in os.listdir(directory):
        if re.match("[a-zA-Z0-9]{8}-[a-zA-Z0-9]{4}-[a-zA-Z0-9]{4}-[a-zA-Z0-9\-]{4}-[a-zA-Z0-9\-]{12}[\.mirbase21\.mirnas\.quantification][\.txt]{0,4}[\.gz]{0,3}",element):
            cfile = element
            print(element)
            return cfile

In [8]:
maxacceptables = 15000
added = 0
for i,cdirectory in enumerate(dirs):
    if re.match("[manifest\.txt]",cdirectory):
        continue
    if "Icon" in cdirectory:
        continue
    cfile = getFilenameFromDir("data/%s"%cdirectory)
    #put header=0 for miRNA!
    #cdf = pd.read_csv(("%s/data/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t', header=0)
    cdf = pd.read_csv(("%s/data/%s/%s"%(working_dir,cdirectory,cfile)), sep='\t')
    #cdf.drop(labels=["reads_per_million_miRNA_mapped","cross-mapped"], axis=1, inplace=True) #miRNA only
    cdf.columns = ["gene", cfile[:]]
    cdf['gene'] = [gene[:15] for gene in cdf['gene']]
    cdf.set_index('gene',inplace=True)
    try:
        df.insert(1,cdf.keys()[0][:],cdf.loc[select_genes,cdf.keys()[0]].values[:60483])
        #df.insert(0,cdf.keys()[0][:],cdf.values.T[0]) #miRNA are not protein-coding
        added=added+1
    except:
        print(sys.exc_info()[0])
    if added >= maxacceptables:
        break
print(added, i)

c5375772-6f28-4bf2-bd70-63a8872b7b7c.htseq.counts.gz


/home/fvalle/anaconda3/envs/phd/lib/python3.6/site-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


36725a42-52bc-4e9a-bb7c-728fbb8b1d19.htseq.counts.gz
b4824c2d-fed2-4fad-bf6f-797119546006.htseq.counts.gz
f95c523a-cfb0-4b7e-9a66-48dcf9d24515.htseq.counts.gz
519e5c0f-8649-4676-8618-79cdc1770de0.htseq.counts.gz
36d40974-4d4f-4726-b98f-1ceab329d0bf.htseq.counts.gz
66b276d6-92c6-4ada-ae69-2ca5db2ad797.htseq.counts.gz
883728dd-ee8e-41ff-9507-1893977ad754.htseq.counts.gz
70151eb2-40c7-4ad1-b4ff-97e1878449fa.htseq.counts.gz
2807d45f-9e15-4e06-8336-de7295fc7509.htseq.counts.gz
7487d66f-3ca5-4df9-ac9e-88f0af7838c3.htseq.counts.gz
7e450615-1252-4b4c-bebb-0f292dec3ceb.htseq.counts.gz
658ec1a1-9948-4be6-a4e2-44ea4655d29c.htseq.counts.gz
95c75be7-d55d-4d74-9179-9be223230a5b.htseq.counts.gz
7133f91c-4128-4578-a1cd-2c1ee81b3f16.htseq.counts.gz
1885eb4c-2149-4746-9c7e-620125b03d94.htseq.counts.gz
e17180f8-d562-485f-ae60-8ae37e898099.htseq.counts.gz
0000772b-773d-4cf8-8baf-0e1e6dbf55e8.htseq.counts.gz
4b5014a3-d76e-485d-b8fa-6c2bbc9ba308.htseq.counts.gz
6533d5fa-8631-4e68-ba3b-2039aa8c96cb.htseq.cou

0a219d18-db73-46e1-a9c9-3565cec06451.htseq.counts.gz
d4d6abaa-9134-42ac-9adf-2045f747abb4.htseq.counts.gz
57b2e9d8-ddae-4457-9959-0d911288e8dc.htseq.counts.gz
bb3e9961-3505-41e8-9d11-7f511c2d9cb1.htseq.counts.gz
0320d3aa-0b8c-4bda-854a-eed5a2d48eda.htseq.counts.gz
0f8ed930-950b-4bae-a02d-e598b74bd82b.htseq.counts.gz
55030097-d293-41e0-a554-38f40282b6dc.htseq.counts.gz
6419a6e7-9e3d-4c0e-9ae9-e14b9ae02661.htseq.counts.gz
f1d5de66-76f3-4911-a877-3571069649a8.htseq.counts.gz
d2fc363b-c14b-441f-9ec6-2a40820d85ab.htseq.counts.gz
4810fb0a-881e-40be-9705-9c41b143e83b.htseq.counts.gz
46927791-e908-49eb-8ad8-21158fc1549a.htseq.counts.gz
54feb6b1-f384-40f5-a680-d0b1764d4d2d.htseq.counts.gz
e7c738b7-08e2-4616-abcb-173382854dbc.htseq.counts.gz
83da7937-9e38-46f0-967a-bcfaeaea1588.htseq.counts.gz
fb13b1f7-2b62-4cd9-8e01-875acc860016.htseq.counts.gz
1ff9e4f4-089a-4364-9ec9-c5ecc6b8c4f6.htseq.counts.gz
c758cf60-68bc-4154-9d43-44339da29889.htseq.counts.gz
f6b9d070-6d11-40f3-9ca0-a20779091825.htseq.cou

5d4c3e4c-0f1b-4e23-af3a-7644cab6c70f.htseq.counts.gz
76e8be2b-99c1-4e5d-97f1-f1829343d76a.htseq.counts.gz
7d7f57fe-a717-4d26-a74b-d050700c57cb.htseq.counts.gz
4b19c0e2-2a61-4f0a-9257-4a528e6b320e.htseq.counts.gz
ce137517-0320-46dd-b4e0-82c02773ba97.htseq.counts.gz
daee50d2-e281-4cbc-ae7b-c52460f4920a.htseq.counts.gz
b667a6ae-353f-4045-a442-a4320231ed73.htseq.counts.gz
450cbf36-7dab-46a4-b6b5-d6b99d51083f.htseq.counts.gz
0a3fb653-84e8-426a-999a-8dc8127dfd6d.htseq.counts.gz
4aa06322-99a1-49e8-ad18-c92cb676da51.htseq.counts.gz
b4100ce6-2507-4ba9-a3d4-b47810c9ba99.htseq.counts.gz
3201fd37-ac44-450b-8a85-058ead558da2.htseq.counts.gz
e9422605-b2da-42b0-aa64-a98472834140.htseq.counts.gz
325a69a6-7312-4c08-94d5-9d76fd913376.htseq.counts.gz
3f500570-271f-4da1-90d5-d60f353521e1.htseq.counts.gz
c7944a7a-c074-43fd-b736-8414bc86467c.htseq.counts.gz
0826fcd7-749e-4788-a81d-f506b9caf7ff.htseq.counts.gz
8b86fbec-ded3-4910-9a7b-c9e9d1c7a377.htseq.counts.gz
2ac83384-33be-4500-b1fa-9d19eb2de6fe.htseq.cou

1943018e-c964-4fd1-b23a-92c1202257b0.htseq.counts.gz
02e47d87-c6c8-4051-a2e6-f65ab814591b.htseq.counts.gz
96ddce60-2be5-4f57-814b-35722aa00fd9.htseq.counts.gz
8b178cb1-d22e-4657-80c6-d7efcddf43a6.htseq.counts.gz
98921931-2f32-470d-97ae-824716d1c034.htseq.counts.gz
b50bf6cf-8540-4fdf-9745-973f1c546f77.htseq.counts.gz
4f6e9a85-31c9-468b-b121-1e235f159010.htseq.counts.gz
b39d696e-8048-41d1-a849-4cccdaf54d06.htseq.counts.gz
87e68da8-743f-40f7-8894-fb66e474bb61.htseq.counts.gz
6c60605b-a647-481e-976d-f2945740faf8.htseq.counts.gz
a720c6bf-6365-4e4b-8761-f5e1097fa711.htseq.counts.gz
71de586e-9fef-4c8e-9c89-1936e9415ab3.htseq.counts.gz
ee7d6b4b-a341-4a9b-9a3a-38526e749cdd.htseq.counts.gz
1f1cef0c-bdbe-41be-bcc1-c4d30bddead4.htseq.counts.gz
cc55afdc-78e2-48c5-b1ac-386033852bca.htseq.counts.gz
01a1c77a-3f93-4561-a079-c771ac054f56.htseq.counts.gz
677dde68-3c06-4e1d-9250-8a99c84643c3.htseq.counts.gz
c6a5effb-1af8-4eab-91cc-9de7780d6249.htseq.counts.gz
a148bcd0-e03e-4d86-a0b3-668cad2a9312.htseq.cou

3fcf11cb-f4ef-4adf-ae8c-f6f225fafebd.htseq.counts.gz
7eed3c23-853b-4d0e-9b2d-81990f2ca0c6.htseq.counts.gz
4dcd18d6-b7b2-4a92-bcbe-361c7278e84e.htseq.counts.gz
9348d9ac-3c90-433e-b5f4-92a7c1287bed.htseq.counts.gz
2e528e8d-46a5-42b0-aa58-d266b30190b0.htseq.counts.gz
e457f8f6-fe2c-4314-a831-c9aa095d4bc7.htseq.counts.gz
ecbe1717-b6b7-4359-b723-3bfc81c82f2d.htseq.counts.gz
488f53a2-92f8-4164-adc8-28b738a312fb.htseq.counts.gz
6069387b-32a3-4e1c-8386-050aa2da67c4.htseq.counts.gz
3c631c4d-ec26-4f29-abcb-bed2221f3da5.htseq.counts.gz
db03f5de-67a4-4503-9742-2ec13cd94ab7.htseq.counts.gz
ceefc80a-4852-44aa-a234-26d0e5062c5f.htseq.counts.gz
ce85344b-0f16-4f96-b8da-af518e2adc71.htseq.counts.gz
432d86b3-b5d0-46c0-96d8-d7d5f4e6c3b6.htseq.counts.gz
98f9dbc0-5a0b-4a9a-b11e-e95ff9ff4497.htseq.counts.gz
c5dde43f-eb24-4d24-92ca-932bb79ea7aa.htseq.counts.gz
780280f7-2ea5-47b9-abbd-c2fc95e63c66.htseq.counts.gz
61f88053-5c64-4141-b6ed-f2a472095b3e.htseq.counts.gz
ea1b4005-545a-4fa3-a3d7-a2dbd1bcc744.htseq.cou

1c421a13-81a2-49de-b0a1-a2c0dc563357.htseq.counts.gz
ef9be0f1-2a17-49b9-b90e-2bb2e4f43c84.htseq.counts.gz
c14af0e0-81e9-43e7-b08d-b751194e35ef.htseq.counts.gz
aa256c1a-55a8-43b1-a553-26444a950087.htseq.counts.gz
cf52b0c5-b1b7-4f0a-b5a1-f1924b39046f.htseq.counts.gz
dc03b9a1-8313-4705-a185-c42ddf9870b7.htseq.counts.gz
c64c76e7-635c-458e-a032-07775aeb03d9.htseq.counts.gz
3ab0577a-0d95-4f3b-9ec8-850820c5ecb3.htseq.counts.gz
be5ceb98-a493-4cad-8bab-281342dea3d0.htseq.counts.gz
24ce4353-ea69-400d-a1e7-a3d785f463f2.htseq.counts.gz
3ab61b3d-13bd-4fca-ac15-f70acbce447c.htseq.counts.gz
552e9a21-3493-4567-8d55-40e7f8009a5f.htseq.counts.gz
db106f15-9af2-4808-ab7b-d1ebb66a2954.htseq.counts.gz
996000bc-080d-4559-80bf-c5059838622a.htseq.counts.gz
ab1f0023-6b52-41b6-9867-87599e4d6d60.htseq.counts.gz
69e11cde-c26b-40d2-a3f5-b668ef590688.htseq.counts.gz
f125d4a1-cb15-4729-9318-433c39c685d7.htseq.counts.gz
a6f657a3-439e-4afd-be04-413f2ed02828.htseq.counts.gz
58050606-3a53-439b-836b-de391c4c630f.htseq.cou

eeff4d01-f161-423f-9d6d-d2d31cc69047.htseq.counts.gz
70cb5052-4284-4147-a745-d2f2b58cb481.htseq.counts.gz
54a62dcd-c9a6-4a97-85c9-62e364766fca.htseq.counts.gz
c1c1111e-858a-438e-8bb6-90df4defcede.htseq.counts.gz
555de98a-5925-41d0-8095-7ae42c480861.htseq.counts.gz
4d0a2c1a-47d5-4798-bacc-c333913c095b.htseq.counts.gz
168e9550-9975-4230-847a-71d868d46fb6.htseq.counts.gz
0a265069-f238-42fe-a1bc-51967ac44f82.htseq.counts.gz
a1a68fe9-9635-4b7f-b9a2-34474ef8c1dc.htseq.counts.gz
456bf061-467d-4bd8-bf71-b0a54ef06dea.htseq.counts.gz
5943b9f4-fd43-46dc-8ebe-a0145e4ee7c3.htseq.counts.gz
45c8b091-bc89-4b52-87d9-2d9f97280abe.htseq.counts.gz
30621023-91a3-4b13-bc10-922491b5f9d3.htseq.counts.gz
b6a39fab-afc2-4498-bb4c-95adeb388c6f.htseq.counts.gz
ef6406a9-591e-421f-8ed4-0fee40f53ef5.htseq.counts.gz
4ce3e99a-a195-414c-a967-7d3795d48934.htseq.counts.gz
1287d36a-6a27-4e4e-a2f7-51727ac3cc0e.htseq.counts.gz
07858399-2ec0-4d01-b197-b362873e56a8.htseq.counts.gz
1cbb4ac2-5588-441a-85f9-59deffb16a22.htseq.cou

In [ ]:
#df = pd.read_csv(("%s/mainTable.csv"%working_dir))

In [ ]:
#df.columns=['gene', 'breast', 'kidney', 'bronchus', 'brain']

In [9]:
df.drop('type_of_gene', inplace=True, axis=1)
df.drop('lenght', inplace=True, axis=1)
df.drop('exons', inplace=True, axis=1)

In [10]:
df.head()

,db709478-5a22-473e-bccb-7f26c3c35578.htseq.counts.gz,ef51ae2e-91ac-4d5f-ba2a-0ae978ba33b9.htseq.counts.gz,6d44a9ba-d0f4-4d8b-b368-036d00198d1f.htseq.counts.gz,b938ae4d-f9cc-42d2-bcf4-5dc14e8fb372.htseq.counts.gz,354bc895-1cf4-41d2-bf30-c52c21369fe2.htseq.counts.gz,14def78d-b065-4d11-9c28-9970f9e065df.htseq.counts.gz,2f01ef65-282d-46a3-acb9-97a27fc30a97.htseq.counts.gz,b4474f7c-f371-4ecb-8c2f-cd7b5d5d243b.htseq.counts.gz,a154c740-f4b4-4b62-a4a8-2273f8f834b1.htseq.counts.gz,ec592b67-6ffb-417e-be69-f5da01020675.htseq.counts.gz,...,2807d45f-9e15-4e06-8336-de7295fc7509.htseq.counts.gz,70151eb2-40c7-4ad1-b4ff-97e1878449fa.htseq.counts.gz,883728dd-ee8e-41ff-9507-1893977ad754.htseq.counts.gz,66b276d6-92c6-4ada-ae69-2ca5db2ad797.htseq.counts.gz,36d40974-4d4f-4726-b98f-1ceab329d0bf.htseq.counts.gz,519e5c0f-8649-4676-8618-79cdc1770de0.htseq.counts.gz,f95c523a-cfb0-4b7e-9a66-48dcf9d24515.htseq.counts.gz,b4824c2d-fed2-4fad-bf6f-797119546006.htseq.counts.gz,36725a42-52bc-4e9a-bb7c-728fbb8b1d19.htseq.counts.gz,c5375772-6f28-4bf2-bd70-63a8872b7b7c.htseq.counts.gz
ENSG00000167578,437.0,481.0,1689.0,609.0,598.0,1450.0,669.0,710.0,1026.0,766.0,...,361.0,1042.0,680.0,2059.0,1019.0,303.0,587.0,717.0,498.0,710.0
ENSG00000078237,417.0,693.0,519.0,316.0,673.0,440.0,603.0,355.0,878.0,567.0,...,427.0,884.0,2288.0,275.0,789.0,523.0,580.0,191.0,390.0,261.0
ENSG00000146083,3045.0,2795.0,5057.0,2580.0,6952.0,1498.0,2677.0,4415.0,6277.0,3646.0,...,5049.0,3442.0,7057.0,3503.0,3175.0,2904.0,3686.0,2489.0,1295.0,2380.0
ENSG00000158486,17.0,34.0,6.0,23.0,29.0,49.0,22.0,324.0,37.0,2.0,...,24.0,8.0,8.0,0.0,120.0,17.0,21.0,116.0,35.0,40.0
ENSG00000198242,43780.0,31528.0,28603.0,16600.0,68636.0,30776.0,16763.0,21617.0,24419.0,24801.0,...,14625.0,19951.0,53871.0,16027.0,26923.0,18869.0,14406.0,22718.0,15216.0,32772.0


In [11]:
print(("genes:%d\trealizations:%d"%(len(df.index),len(df.columns))))

genes:19369	realizations:1003


In [12]:
df.to_csv(("%s/mainTable.csv"%working_dir), index=True)

expression per gene

In [ ]:
np.sum(df.loc[0,:].values[1:])

expression per case

In [ ]:
np.sum(df.loc[:,df.keys()[1]].values)

In [ ]:
df_files=pd.read_csv("files.dat",header=0,index_col=0)
df_files[df_files.index.isin(df.columns)].to_csv("files.dat",index=True,header=True)

## Zipf

In [ ]:
A = [df.loc[:,df.keys()[9]].values, df.loc[:,df.keys()[45]].values, df.loc[:,df.keys()[42]].values]
colors = ['red', 'blue', 'yellow']
fig = plt.figure()
x = np.arange(1, len(A[0]))
plt.ylabel("frequencyi")
plt.xlabel("RANKi")
plt.yscale('log')
plt.xscale('log')
plt.plot(x, 1./x, 'g--', label='$r^{-\\alpha}$')
for i,a in enumerate(A):
    plt.plot(np.sort(a/np.sum(a, dtype=float))[::-1], c=colors[i], label = 'file') 
plt.legend(fontsize=16)
plt.show()

## Zipf per cancer

In [ ]:
colors = ['red', 'blue', 'yellow', 'green']
fig = plt.figure(figsize=(8,6))
x = np.logspace(0, 5)
plt.ylabel("$frequency_i$", fontsize=16)
plt.xlabel("$RANK_i$", fontsize=16)
plt.yscale('log')
plt.xscale('log')
plt.plot(x, 1./x, 'g--', label='$r^{-1}$')
for i,k in enumerate(df.keys()[1:]):
    a=df[k]
    plt.plot(np.sort(a/np.sum(a))[::-1], c=colors[i], label = k) 
plt.legend(fontsize=16)
plt.show()

In [ ]:
fig.savefig("zipfPerCancer.pdf")

# gene table

In [ ]:
genes = open("%s/genes.txt"%working_dir, "w")
for gene in cdf['gene'].values:
    genes.write("%s\n"%gene)
genes.close()

# table as image

In [ ]:
from PIL import Image

In [ ]:
bits = np.array([df.loc[i,:].values[1:] for i in np.arange(len(df['gene']))])

In [ ]:
bits.shape

In [ ]:
img = Image.fromarray(bits, mode='1')

In [ ]:
img.save("mat.bmp")